In [ ]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# Launch MAPDL
mapdl = launch_mapdl(
    loglevel="ERROR",
    exec_file="C:/Program Files/ANSYS Inc/v232/ansys/bin/winx64/ANSYS232.exe",
    nproc=10
)
mapdl.clear()
mapdl.prep7()
mapdl.units("SI")  # Use SI units

# --- Define Material 1: Concrete (temperature-dependent) ---
mapdl.mp("DENS", 1, 2400)  # Density

# Specific heat C defined at 100, 200, 400, 500°C
mapdl.mptemp(1, 20)
mapdl.mptemp(2, 100)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 400)
mapdl.mptemp(5, 500)
mapdl.mpdata("C", 1, 1, 900)
mapdl.mpdata("C", 1, 2, 900)
mapdl.mpdata("C", 1, 3, 1000)
mapdl.mpdata("C", 1, 4, 1100)  
mapdl.mpdata("C", 1, 5, 1100)  

# Thermal conductivity KXX defined at 100, 400, 500°C, using linear formula k = 2.5 - (T - 100)/600
def k_formula(T): return 2.5 - (T - 100)/600
mapdl.mptemp(1, 100)
mapdl.mptemp(2, 150)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 300)
mapdl.mptemp(5, 400)
mapdl.mpdata("KXX", 1, 1, k_formula(100))  # 2.5
mapdl.mpdata("KXX", 1, 2, k_formula(150))  # 2.4166667
mapdl.mpdata("KXX", 1, 3, k_formula(200))  # 2.3333333
mapdl.mpdata("KXX", 1, 4, k_formula(300))  # 2.1666667
mapdl.mpdata("KXX", 1, 5, k_formula(400))  # 2

# --- Define Material 2: Structural Steel (constant properties) ---
mapdl.mp("DENS", 2, 7850)
mapdl.mp("C", 2, 434)
mapdl.mp("KXX", 2, 60.5)

# --- Geometry parameters ---
od_media = 0.254     # Outer diameter of surrounding medium (m)
od_hx = 0.0508       # Outer diameter of heat exchanger tube (m)
t_hx = 0.00635       # Tube wall thickness (m)
id_hx = od_hx - 2 * t_hx  # Inner diameter of heat exchanger tube (m)
length = 0.508       # Height (m)
num_radial = 10      # Number of radial mesh divisions
num_circ = 9         # Number of circumferential mesh divisions

# --- Create Keypoints ---
k0 = mapdl.k("", 0, 0, 0)
inner_kps = []
outer_kps = []
for i in range(num_circ):
    angle = i * (90 / num_circ)
    x_inner = (id_hx / 2) * np.cos(np.deg2rad(angle))
    y_inner = (id_hx / 2) * np.sin(np.deg2rad(angle))
    x_outer = (od_media / 2) * np.cos(np.deg2rad(angle))
    y_outer = (od_media / 2) * np.sin(np.deg2rad(angle))
    inner_kps.append(mapdl.k("", x_inner, y_inner, 0))
    outer_kps.append(mapdl.k("", x_outer, y_outer, 0))

# --- Create Lines and Areas ---
inner_arcs = [mapdl.l(inner_kps[i], inner_kps[(i+1)%num_circ]) for i in range(num_circ)]
outer_arcs = [mapdl.l(outer_kps[i], outer_kps[(i+1)%num_circ]) for i in range(num_circ)]
radial_lines = [mapdl.l(inner_kps[i], outer_kps[i]) for i in range(num_circ)]
areas = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    lines = [inner_arcs[i], radial_lines[next_i], outer_arcs[i], radial_lines[i]]
    areas.append(mapdl.al(*lines))

# --- Set element type and mesh controls ---
mapdl.et(1, "SOLID70")
volumes = [mapdl.vext(a, dz=length) for a in areas]

# Set radial mesh size
mapdl.lsel("S", "LINE", "", min(radial_lines), max(radial_lines))
mapdl.lesize("ALL", "", "", num_radial, 1)

# Set axial mesh size
num_z = 10
mapdl.lsel("S", "LOC", "X", 0)
mapdl.lsel("R", "LOC", "Y", 0)
mapdl.lsel("R", "LOC", "Z", 0, length)
mapdl.lesize("ALL", "", "", num_z, 1)
mapdl.allsel()
mapdl.vmesh("ALL")
mapdl.smrtsize(2)

# --- Plot Mesh ---
mapdl.eplot(vtk=True, show_edges=True, line_width=1)


C:\Users\arnold\AppData\Local\Temp\ipykernel_43208\1160459773.py:85: DeprecationWarning: From 0.70.0, the arguments 'vtk' and 'use_vtk' are deprecated. To use interactive plots, use 'graphics_backend=GraphicsBackend.PYVISTA' instead. 
  mapdl.eplot(vtk=True, show_edges=True, line_width=1)


[82, 87, 110]


Widget(value='<iframe src="http://localhost:64415/index.html?ui=P_0x19ac03fc190_1&reconnect=auto" class="pyvis…

In [ ]:
mapdl.finish()
mapdl.slashsolu()

import matplotlib.pyplot as plt

# --- Set Up Transient Thermal Analysis ---
mapdl.antype(4)  # Transient thermal analysis
mapdl.trnopt("FULL")  # Full transient solution
mapdl.kbc(0)  # Ramped boundary conditions (smooth application)
mapdl.timint("ON")  # Enable time integration

# --- Define Initial Conditions ---
mapdl.ic("ALL", "TEMP", 100)  # Set initial temperature to 100°C for all nodes

# --- Define Time Steps ---
time_steps = np.linspace(0, 14400, 1441)  # From 0 to 14400 seconds in 10s steps
temp_load = np.linspace(100, 400, len(time_steps))  # Temperature linearly increases from 100°C to 400°C

for i, t in enumerate(time_steps):
    if i == 0:
        continue  # Skip initial step (already defined by initial condition)

    mapdl.time(t)

    # --- Select Nodes Near Inner Wall ---
    inner_radius = id_hx / 2  # Inner radius of the pipe
    points = mapdl.mesh.grid.points
    radii = np.sqrt(points[:, 0]**2 + points[:, 1]**2)

    # Find nodes with radius close to inner_radius (within tolerance)
    final_node_indices = np.where(np.abs(radii - inner_radius) < 1e-3)[0]
    final_nodes = final_node_indices + 1  # Node numbers start from 1 in MAPDL

    # Clear current selection and select target nodes
    mapdl.nsel("NONE")
    for node in final_nodes:
        mapdl.nsel("A", "NODE", "", node)

    # Optional: Plot nodes at first time step for verification
    # if i == 1:
    #     mapdl.nplot(vtk=True, show_node_numbering=True)

    # --- Apply Temperature Boundary Condition to Selected Nodes ---
    mapdl.d("ALL", "TEMP", temp_load[i])

    # --- Restore Selection ---
    mapdl.nsel("ALL")

    # --- Time Step Settings ---
    mapdl.autots("ON")  # Enable automatic time stepping
    mapdl.deltim(10)    # Suggested time increment

    # --- Solve at Current Time Step ---
    mapdl.solve()

# --- End Solution Phase ---
mapdl.finish()

# --- Enter Post-Processing ---
mapdl.post1()


*****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

          ***** MAPDL RESULTS INTERPRETATION (POST1) *****

 *** NOTE ***                            CP =       0.000   TIME= 00:00:00
 Reading results into the database (SET command) will update the current 
 displacement and force boundary conditions in the database with the     
 values from the results file for that load set.  Note that any          
 subsequent solutions will use these values unless action is taken to    
 either SAVE the current values or not overwrite them (/EXIT,NOSAVE).    


 USE LAST SUBSTEP ON RESULT FILE  FOR LOAD CASE 0 

 SET COMMAND GOT LOAD STEP=  1440  SUBSTEP=     1  CUMULATIVE ITERATION=  1440
   TIME/FREQUENCY=  14400.    
 TITLE=

In [ ]:
mapdl.finish()
mapdl.post1()

inner_radius = id_hx / 2
outer_radius = od_media / 2

# Select nearest to (inner_radius, 0, z_center)
mapdl.nsel("S", "LOC", "X", inner_radius - 1e-3, inner_radius + 1e-3)
mapdl.nsel("R", "LOC", "Y", -1e-3, 1e-3)
mapdl.nsel("R", "LOC", "Z", -1e-3, 1e-3)
inner_node = int(mapdl.get("INNER", "NODE", 0, "NUM", "MAX"))

# Select nearest to (outer_radius, 0, z_center)
mapdl.nsel("S", "LOC", "X", outer_radius - 1e-3, outer_radius + 1e-3)
mapdl.nsel("R", "LOC", "Y", -1e-3, 1e-3)
mapdl.nsel("R", "LOC", "Z", -1e-3, 1e-3)
outer_node = int(mapdl.get("OUTER", "NODE", 0, "NUM", "MAX"))

mapdl.allsel()
mapdl.finish()

print(f"inner_node = {inner_node}, outer_node = {outer_node}")


inner_node = 1, outer_node = 15


In [13]:
grid = mapdl.mesh.grid  # PyVista-compatible mesh
nodes = mapdl.mesh.nodes
coords = grid.points  # Nx3 array of (x, y, z)


for nid, coord in zip(nodes, coords):
    print(f"Node {nid}: x={coord[0]:.4f}, y={coord[1]:.4f}, z={coord[2]:.4f}")


Node [0.01905 0.      0.     ]: x=0.0190, y=0.0000, z=0.0000
Node [0.01876059 0.003308   0.        ]: x=0.0188, y=0.0033, z=0.0000
Node [0.01895353 0.00110267 0.        ]: x=0.0190, y=0.0011, z=0.0000
Node [0.01885706 0.00220533 0.        ]: x=0.0189, y=0.0022, z=0.0000
Node [0.12507058 0.02205332 0.        ]: x=0.1251, y=0.0221, z=0.0000
Node [0.02939159 0.00518253 0.        ]: x=0.0294, y=0.0052, z=0.0000
Node [0.04002259 0.00705706 0.        ]: x=0.0400, y=0.0071, z=0.0000
Node [0.05065359 0.00893159 0.        ]: x=0.0507, y=0.0089, z=0.0000
Node [0.06128459 0.01080613 0.        ]: x=0.0613, y=0.0108, z=0.0000
Node [0.07191559 0.01268066 0.        ]: x=0.0719, y=0.0127, z=0.0000
Node [0.08254659 0.01455519 0.        ]: x=0.0825, y=0.0146, z=0.0000
Node [0.09317759 0.01642972 0.        ]: x=0.0932, y=0.0164, z=0.0000
Node [0.10380859 0.01830425 0.        ]: x=0.1038, y=0.0183, z=0.0000
Node [0.11443958 0.02017879 0.        ]: x=0.1144, y=0.0202, z=0.0000
Node [0.127 0.    0.   ]: x=0

In [ ]:
import os

mapdl.post1()
num_steps = len(time_steps)
inner_temps = []
outer_temps = []
for i in range(1, num_steps):
    mapdl.set(i)
    temp_inner = mapdl.get("TIN", "NODE", inner_node, "TEMP")
    temp_outer = mapdl.get("TOUT", "NODE", outer_node, "TEMP")
    inner_temps.append(temp_inner)
    outer_temps.append(temp_outer)

import pandas as pd

df = pd.DataFrame({
    "Time (s)": time_steps[1:],  
    "Inner Temp (°C)": inner_temps,
    "Outer Temp (°C)": outer_temps,
})

df.to_csv("temperature_points.csv", index=False)
print("Saved temperature record to 'temperature_points.csv'")

Saved temperature record to 'temperature_points.csv'
